In [192]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from dash import Dash, html, dcc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

from PIL import Image

import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template


# Input and cleaning data
sales = pd.DataFrame()

files = os.listdir('./Sales_Data')
for file in files :
    df = pd.read_csv(f'./Sales_Data/{file}')
    sales = pd.concat([sales, df])
sales = sales[~(sales['Order ID'] == 'Order ID')]
sales.dropna(inplace=True)

sales = sales.copy()
sales['Order ID'] = sales['Order ID'].astype(int)
sales['Quantity Ordered'] = sales['Quantity Ordered'].astype(int)
sales['Price Each'] = sales['Price Each'].astype(float)
sales['Date'] = pd.to_datetime(sales['Order Date'].apply(lambda x: x[:8]))
sales['Sales Amount'] = sales['Quantity Ordered'] * sales['Price Each']

sales['State']  = sales['Purchase Address'].apply(lambda x: x.split(', ')[1])
sales['Start Of Month'] = sales['Date'].apply(lambda x: x.replace(day=1))
sales['week'] = sales['Date'].dt.isocalendar().week
sales['year-week'] = sales.apply(
    lambda x: str(x['Date'].year) + '-' + ('0' if len( str( x['week'] ) ) < 2 else '') + str(x['week'])
, axis=1)
sales = sales[~(sales['Date'] == '2020-1-1')]
sales.rename(columns={'Quantity Ordered':'Sales Quantity'}, inplace=True)

# Sales target
sales_kpi = pd.DataFrame({
    'Month': list(sales['Start Of Month'].sort_values().unique()),
    'Amount Target': [1800000, 2300000, 2500000, 3600000, 3200000, 2600000, 2600000, 2600000, 2000000, 3600000, 3200000, 5000000],
    'Unit Target': [11000, 12000, 18000, 22000, 20000, 18000, 13000, 12800, 22000, 22000, 19000, 26000]
})

# monthly_sales dataframe
monthly_sales = sales.groupby('Start Of Month', as_index=False)[['Sales Amount', 'Sales Quantity']].sum()
monthly_sales = monthly_sales.merge(sales_kpi, how='left', left_on='Start Of Month', right_on='Month').drop('Month', axis=1)
monthly_sales['Amount to Target'] = (monthly_sales['Sales Amount'] / monthly_sales['Amount Target'])*100
monthly_sales['Amount to Target'] = monthly_sales['Amount to Target'].apply(lambda x: round(x, 0))
monthly_sales['Unit to Target'] = (monthly_sales['Sales Quantity'] / monthly_sales['Unit Target'])*100
monthly_sales['Unit to Target'] = monthly_sales['Unit to Target'].apply(lambda x: round(x, 0))
monthly_sales['Previous Month'] = monthly_sales['Sales Amount'].shift()
monthly_sales['Growth rate'] = monthly_sales['Sales Amount']/monthly_sales['Previous Month']*100 - 100
monthly_sales['Growth rate'] = monthly_sales['Growth rate'].apply(lambda x: round(x, 1))

# Weekly_sales dataframe
weekly_sales = sales.groupby('year-week', as_index=False)[['Sales Amount', 'Sales Quantity']].sum()
# Monthly_sale_by_state dataframe
monthly_sale_by_state = sales.groupby(['Start Of Month', 'State'], as_index=False)[['Sales Amount', 'Sales Quantity']].sum()



# Monthly Sales Trend
fig = px.bar(monthly_sales,
        x= 'Start Of Month',
        y='Sales Amount', 
    )

fig.update_layout(
    plot_bgcolor=None, paper_bgcolor=None,
    title ={
        'text': 'Monthly Sales Trend',
        'x':0.5,
    },
    xaxis = {'tickvals': monthly_sales['Start Of Month']},
)
fig.update_xaxes(title='Month')
fig.update_yaxes(title='Monthly Sales')
fig.update_traces(marker_color='#000000')

fig.add_trace(
    go.Scatter(
        x=monthly_sales['Start Of Month'],
        y=monthly_sales['Amount Target'],
        mode='markers',
        name='Sales Target',
        marker={'color':'red'}
    )
)

# Weekly sales trend
fig2 = px.line(weekly_sales,
        x= 'year-week',
        y='Sales Amount',
    )

fig2.update_layout(
    plot_bgcolor='#FFFFFF', paper_bgcolor='#FFFFFF',
    title ={
        'text': 'Weekly Sales Trend',
        'x':0.5,
    }
)
fig2.update_xaxes(title='Week')
fig2.update_yaxes(title='Weekly Sales')
fig2.update_xaxes(type='category')
fig2.update_traces(line_color='#000000')

# App
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = Dash(__name__, external_stylesheets=[dbc.themes.SKETCHY, dbc_css])

load_figure_template('SKETCHY')
pil_image = Image.open("logo (1).png")

app.layout = dbc.Container([
    # First Row : Logo and Title
    dbc.Row([
        dbc.Col(html.Img(src=pil_image, height=100, width=100)
                , width=2 ),
        dbc.Col(html.H2('SUPERSTORE SALES DASHBOARD', style={'font-size': '50px', 'font-family':'Segoe UI', 'font-weight':'bold', 'text-align':'center'}), width=10)
    ]),
    
    # Second Row: Drop down and 3 kpi cards
    dbc.Row([
        dbc.Col(dbc.Card(
            dcc.Dropdown(
                id='Month Dropdown',
                options= [  {'label': 'Jan-2019' , 'value' : pd.Timestamp('2019-01-01 00:00:00')},
                            {'label': 'Feb-2019' , 'value' : pd.Timestamp('2019-02-01 00:00:00')},
                            {'label': 'Mar-2019' , 'value' : pd.Timestamp('2019-03-01 00:00:00')},
                            {'label': 'Apr-2019' , 'value' : pd.Timestamp('2019-04-01 00:00:00')},
                            {'label': 'May-2019' , 'value' : pd.Timestamp('2019-05-01 00:00:00')},
                            {'label': 'Jun-2019' , 'value' : pd.Timestamp('2019-06-01 00:00:00')},
                            {'label': 'Jul-2019' , 'value' : pd.Timestamp('2019-07-01 00:00:00')},
                            {'label': 'Aug-2019' , 'value' : pd.Timestamp('2019-08-01 00:00:00')},
                            {'label': 'Sep-2019' , 'value' : pd.Timestamp('2019-09-01 00:00:00')},
                            {'label': 'Oct-2019' , 'value' : pd.Timestamp('2019-10-01 00:00:00')},
                            {'label': 'Nov-2019' , 'value' : pd.Timestamp('2019-11-01 00:00:00')},
                            {'label': 'Dec-2019' , 'value' : pd.Timestamp('2019-12-01 00:00:00')}
                         ],
                value= pd.Timestamp('2019-12-01 00:00:00'), 
                style={'height': '30px'}
            )
        )),
        dbc.Col(dbc.Card(dbc.CardBody([
            dcc.Markdown('**Total Revenue**', style={'text-align':'center', 'font-weight':'bold'}),
            html.H3(id='kpi revenue', style={'text-align':'center', 'font-weight':'bold', 'font-size': 50}),
            html.H3(id='revenue note', style={'text-align':'center', 'font-weight':'bold', 'font-size': 15})
        ], style={'height': '140px'}))),
        dbc.Col(dbc.Card(dbc.CardBody([
            dcc.Markdown('**Unit Sold**', style={'text-align':'center', 'font-weight':'bold'}),
            html.H3(id='kpi unit', style={'text-align':'center', 'font-weight':'bold', 'font-size': 50}),
            html.H3(id='unit note', style={'text-align':'center', 'font-weight':'bold', 'font-size': 15})
        ], style={'height': '140px'}))),
        dbc.Col(dbc.Card(dbc.CardBody([
            dcc.Markdown('**Growth Rate**', style={'text-align':'center', 'font-weight':'bold'}),
            html.H3(id='kpi growth', style={'text-align':'center', 'font-weight':'bold', 'font-size': 50})
        ], style={'height': '140px'}))),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col(dcc.Graph(id='monthlysale-graph', figure=fig, style={'height':'620px'})),
        dbc.Col([
            dbc.Row(dcc.Graph(id='weeklysale-graph', figure=fig2, style={'height':'310px'})),
            dbc.Row(dcc.Graph(id='monthly sale by state'),  style={'height':'310px'})
        ])
    ])
    
], style={"backgroundColor": "#f0f0f0"})

@app.callback(
    Output('monthly sale by state', 'figure'),
    Output('kpi revenue', 'children'),
    Output('kpi unit', 'children'),
    Output('kpi growth', 'children'),
    Output('revenue note','children'),
    Output('unit note','children'),
    Input('Month Dropdown','value')
)

def plot_charts(month):
    if not month:
        raise PreventUpdate
    data = monthly_sale_by_state.query("`Start Of Month` == @month")
    
    fig = px.bar(
        data,
        x='Sales Amount',
        y='State'
    )
    
    sales = round(monthly_sales.query("`Start Of Month` == @month").iloc[0, 1]/ 1000000,1)
    sales_export = f'{sales} M'
    
    unit = monthly_sales.query("`Start Of Month` == @month").iloc[0, 2]
    unit_export = f'{unit:,.0f}'
    
    growth = monthly_sales.query("`Start Of Month` == @month").iloc[0,-1]
    growth_export = f'{growth} %'
    
    amount_to_target = monthly_sales.query("`Start Of Month` == @month").iloc[0,5]
    amount_to_target_export = f'Achieved : {int(amount_to_target)} %'
    
    unit_to_target = monthly_sales.query("`Start Of Month` == @month").iloc[0,6]
    unit_to_target_export = f'Achieved : {int(unit_to_target)} %'
    
    return fig, sales_export, unit_export, growth_export, amount_to_target_export, unit_to_target_export

if __name__ == "__main__":
    app.run_server(debug=True, 
                   jupyter_mode='tab')

C:\Users\Phat\AppData\Local\Temp\ipykernel_20276\18300458.py:33: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>